# Projeto Covid-19

## DIO

Primeiro vamos importar algumas das bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
#vamos importar os dados para o projeto
url = 'covid_19_data.csv'

In [ ]:
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])

In [ ]:
df

In [ ]:
# Conferir os tipos de cada coluna
df.dtypes

Nomes de colunas não devem ter letras maiúsculas e nem caracteres especiais. Vamos implementar uma função para fazer a limpeza dos nomes dessas colunas

In [ ]:
import re

def corrige_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name).lower()

In [ ]:
corrige_colunas("SADAsd/ asdassdADS") #testando

In [ ]:
# Vamos corrigir todas as colunas do df

df.columns = [corrige_colunas(col) for col in df.columns]

In [ ]:
df

# Brasil

vamos selecionar apenas os dados do Brasil para investigar

In [ ]:
df.loc[df.countryregion == "Brazil"]

In [ ]:
brasil = df.loc[
    (df.countryregion== 'Brazil') &
    (df.confirmed > 0)
]

In [ ]:
brasil

# Casos confirmados

In [ ]:
# Gráfico da evolução de casos confirmados
px.line(brasil, 'observationdate', 'confirmed', title='Casos confirmados no Brasil')

# Novos casos por dia

In [ ]:
# Técnica de programação funcional

brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))

In [ ]:
brasil

In [ ]:
px.line(brasil, x='observationdate', y='novoscasos', title="Novos casos por dia")

# Mortes

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=brasil.observationdate,y=brasil.deaths, name="Mortes",
              mode='lines+markers', line={'color':'red'})
)

#layout
fig.update_layout(title="Mortes por COVID-19 no Brasil")

fig.show()

# Taxa de crescimento

taxa_crescimento = {presente/passado}** (1/n)-1

In [ ]:
def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
    # Se data inicio for None, define como a primeira data disponível
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable]>0].min() 
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    if data_fim == None:
        data_fim = data.observationdate.iloc[-1]
    else:
        data_fim = pd.to_datetime(data_fim)
        
    # Define os valores do presente e passado
    passado = data.loc[data.observationdate == data_inicio, variable].values[0]
    presente = data.loc[data.observationdate == data_fim, variable].values[0]
    
    # Define o número de pontos no tempo que vamos avaliar
    n = (data_fim - data_inicio).days
    
    # Calcular a taxa
    taxa = (presente/passado)**(1/n)-1
    
    return taxa*100

In [ ]:
# Taxa de crescimento médio do COVID no Brasil em todo período
taxa_crescimento(brasil, 'confirmed')

In [ ]:
def taxa_crescimento_diaria(data, variable, data_inicio=None):
    # Se data inicio for None, define como a primeira data disponível
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable]>0].min() 
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    data_fim = data.observationdate.max()
    # Define o número de pontos no tempo que vamos avaliar
    n=(data_fim - data_inicio).days
    
    # Taxa calculada de um dia para o outro
    taxas= list(map(
        lambda x: (data[variable].iloc[x]- data[variable].iloc[x-1]) / data[variable].iloc[x-1],
        range(1, n+1)
    ))
    
    return np.array(taxas)*100

In [ ]:
tx_dia = taxa_crescimento_diaria(brasil, 'confirmed')

In [ ]:
tx_dia

In [ ]:
primeiro_dia = brasil.observationdate.loc[brasil.confirmed>0].min()

px.line(x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:],
       y=tx_dia, title="Taxa de crescimento de casos confirmados no Brasil")

# Predições

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [ ]:
confirmados = brasil.confirmed
confirmados.index = brasil.observationdate
confirmados

In [ ]:
res = seasonal_decompose(confirmados)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show()

# ARIMA

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima

modelo = auto_arima(confirmados)

In [112]:
fig = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados, name="Observados"
))

fig.add_trace(go.Scatter(
    x=confirmados.index, y=modelo.predict_in_sample(), name="Preditos"
))

fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-20', '2020-06-20'), y=modelo.predict(31), name="Forecast"
))

fig.update_layout(title = 'Previsão de casos confirmados no Brasil para os próximos 30 dias')
fig.show()

# Modelo de crescimento

Vamos usar a biblioteca fbprophet

In [3]:
!conda install -c conda-forge fbprophet -y

^C


In [2]:
from fbprophet import Prophet

ModuleNotFoundError: No module named 'fbprophet'

In [110]:
# Preprocessamentos

train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5:]

# Renomeando colunas
train.rename(columns={'observationdate':'ds', 'confirmed': 'y'}, inplace=True)
test.rename(columns={'observationdate':'ds', 'confirmed': 'y'}, inplace=True)

# Definir o modelo de crescimento
profeta = Prophet(growth='logistic', changepoints=['2020-03-21','2020-03-30','2020-04-25','2020-05-03','2020-05-10'])





#pop = 211463256
pop = 1000000
train['cap']=pop

#Treina o modelo
profeta.fit(train)

# Construir previsões para o futuro
future_dates = profeta.make_future_dataframe(periods=200)
futura_dates['cap']=pop
forecast = profeta.predict(futura_dates)

2

In [ ]:
ig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))
fig.add_trace(go.Scatter(x=train.ds, y=train.y, name='Observados - Treino'))
fig.update_layout(title='Predições de casos confirmados no Brasil')
fig.show()